### create new features for first active month variable

#### The training set has the first active month information, we can create new variable from it, for each card_id try to extract two transaction date to determine the lag for each id 


In [30]:
import pandas as pd
import numpy as np

#hist_tran=pd.read_csv('training_split_by_us_transaction.csv')
#trans_date=hist_tran[["card_id",'month_lag',"purchase_date"]]
#trans_date.to_csv('Elo Project [feature creation2]cardID+month_lag+purchase_date .csv')

In [31]:
train=pd.read_csv('train_card_info.csv')
trans_date=pd.read_csv('Elo Project [feature creation2]cardID+month_lag+purchase_date .csv',index_col='Unnamed: 0')

C:\Anacon\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [32]:
trans_date.head()
trans_date.sort_values(by=['card_id'],inplace=True)
unique_id=trans_date.card_id.unique().tolist()
trans_date.set_index('card_id',inplace=True)
trans_date.head()

,month_lag,purchase_date
card_id,,
C_ID_00007093c1,-10,2017-04-17 19:19:54
C_ID_00007093c1,-5,2017-09-25 20:07:44
C_ID_00007093c1,-4,2017-10-28 20:11:04
C_ID_00007093c1,-1,2018-01-08 18:13:40
C_ID_00007093c1,-6,2017-08-28 11:46:28


In [33]:
#draw 5 records from each ID
draw=[]
for i in unique_id:
    draw.append(trans_date.loc[i][:3])

len(draw) #161533

161533

In [34]:
from datetime import datetime
from dateutil.relativedelta import relativedelta


reference_date1={}

for c in range(len(draw)):
    newtrans_date1=[]
    t_lst=draw[c].purchase_date.tolist()
    lag=draw[c].month_lag.tolist()

    for i in range(len(t_lst)):
        newtrans_date1.append(t_lst[i].split(" ")[0].split("-")+[lag[i]])
        
    reference_date=[]
    
    for i in range(len(newtrans_date1)):
        y,m,d,l=[int(t) for t in newtrans_date1[i]]
        t=datetime(y, m, d) - relativedelta(months=l)
        reference_date.append(t.strftime('%Y-%m'))
    
    reference_date1[draw[c].index[0]]=reference_date

In [35]:
#all the reference date for random draw 3 records belong to the same id are the same 
#save only one date 
reference_date2={}
for i in unique_id:
    if len(np.unique(reference_date1[i]))!= 1:
        print(i)
    else: 
        reference_date2[i]=reference_date1[i][0]
        

In [36]:
#double check if all the index is unique
len(train.card_id) ==len(train.card_id.unique())

True

In [37]:
train.set_index('card_id',inplace=True)

In [38]:
active_m_lag=[]
for i in train.index.tolist():
    f_a_m = datetime.strptime(train.first_active_month.loc[i],'%Y-%m')
    r_m = datetime.strptime(reference_date2[i],'%Y-%m')
    if relativedelta(f_a_m,r_m).years==0:
        active_m_lag.append(relativedelta(f_a_m,r_m).months)
    else:
        active_m_lag.append(relativedelta(f_a_m,r_m).years*12+relativedelta(f_a_m,r_m).months)

In [39]:
#it seems to be more than 6 years lag between the first active month and the reference date
np.unique(active_m_lag)

array([-75, -74, -73, -72, -70, -69, -68, -67, -66, -65, -64, -63, -62,
       -61, -60, -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49,
       -48, -47, -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36,
       -35, -34, -33, -32, -31, -30, -29, -28, -27, -26, -25, -24, -23,
       -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10,
        -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1])

In [40]:
pd.Series(active_m_lag).value_counts().head()

-3    13329
-4    13218
-5    12573
-6    11613
-2    11358
dtype: int64

In [41]:
#examin the code
active_m_lag.index(-75)
train.loc[train.index[21458]]
reference_date2[train.index[21458]]

'2018-02'

In [42]:
#decide to apply year lag instead of month lag

import math

active_y_lag=[]
for i in train.index.tolist():
    f_a_m = datetime.strptime(train.first_active_month.loc[i],'%Y-%m')
    r_m = datetime.strptime(reference_date2[i],'%Y-%m')
    if relativedelta(f_a_m,r_m).years==0:
        active_y_lag.append(relativedelta(f_a_m,r_m).years)
        
    # I decide to bin the year lag>5 to 5 so year lag 5 means =>5
    else:
        year_lag=math.floor((relativedelta(f_a_m,r_m).years*12+relativedelta(f_a_m,r_m).months)/12)   
        if year_lag <= -5:
            active_y_lag.append(-5)
        else:
            active_y_lag.append(year_lag)

In [43]:
# for future reference
pd.Series(active_y_lag).value_counts()

 0    101337
-2     39078
-3     10712
-1      5831
-4      3514
-5      1061
dtype: int64

In [46]:
#bin the year lag>5 to 5 so year lag 5 means =>5
#after binning

pd.Series(active_y_lag).value_counts()

 0    101337
-2     39078
-3     10712
-1      5831
-4      3514
-5      1061
dtype: int64

#### merge the new feature to training set

In [45]:
train.drop(['first_active_month'],axis=1,inplace=True)
year_lag_df=pd.DataFrame(pd.Series(active_y_lag,train.index),columns=['first_active_month_year_lag_to_reference_date'])
new_train=pd.concat([train,year_lag_df],axis=1)


categorical=new_train[['feature_1','feature_2','feature_3','first_active_month_year_lag_to_reference_date']]
categorical=categorical.astype({'feature_1':'category','feature_2':'category','feature_3':'category','first_active_month_year_lag_to_reference_date':'category'})

categorical.to_csv('Elo Project [feature creation2] [No transform] Training Set Part1.csv')

In [12]:
new_train.head()

,feature_1,feature_2,feature_3,target,first_active_month_year_lag_to_reference_date
card_id,,,,,
C_ID_66645317a4,3,1,1,0.063921,-2
C_ID_716e986eb1,3,2,1,-0.932504,0
C_ID_ed55d4da27,2,2,0,-1.358110,0
C_ID_66b5e358b5,3,3,1,-0.108409,0
C_ID_79bfa65fcf,5,1,1,-0.068852,0


In [241]:
new_train.feature_1.value_counts()

3    58812
2    44662
5    32377
4    16018
1     9664
Name: feature_1, dtype: int64

In [242]:
new_train.feature_2.value_counts()

1    71364
2    59925
3    30244
Name: feature_2, dtype: int64

In [243]:
new_train.feature_3.value_counts()

1    91189
0    70344
Name: feature_3, dtype: int64

In [271]:

categorical=pd.get_dummies(categorical)
#len(categorical.columns)#16

In [277]:
final_train=pd.concat([categorical,new_train.target],axis=1)
#final_train.to_csv('C:/Users/Rai Chiang/Documents/Depaul Course/672/data/Elo Project [feature creation]/Elo Project [feature creation2] final Training Set.csv')